1. Open the transcript and get all span element
2. Put it into a dataframe, second / value
3. Tokenize it, if it is more than two elements then expand it and add new elements after the location
4. Open the original folia file check if number of tokens equals to the number in the df
5. If so identify the sentence beginning

In [60]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
%config Completer.use_jedi = False
import utils.db as db
import glob
import datetime
import pandas as pd
import requests
import json

In [41]:
testimony_id = 25651
files = glob.glob("data/transcripts/"+str(testimony_id)+".*.xml")
files_ordered = {int(element.split('.')[-2]):element for element in files}
files_order = sorted(files_ordered)
files_ordered_final = [files_ordered[element] for element in files_order]
all_spans = []
for file in files_ordered_final:
    p1 = minidom.parse(file)
    spans = p1.getElementsByTagName('span')
    all_spans.extend(spans)

In [47]:
    data = [{'time':datetime.timedelta(milliseconds=int(element.getAttribute('m'))).seconds,'value':element.firstChild.nodeValue} for element in all_spans]

In [109]:
df = pd.DataFrame(data)
new_list = []

In [110]:
for element in df.iterrows():
    label = element[1].value
    if label == None:
        continue
    props = {'annotators': 'tokenize'}
    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                        json.dumps(props)},
                        data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    for value in result['tokens']:
        try:
            new_row={'time':element[1].time,'value':value['word']}
            new_list.append(new_row)
        except:
            pass

In [111]:
len(new_list)

73016

In [108]:
new_list[-2]

{'time': 1649, 'value': 'yes'}

In [98]:
all_spans[4917].firstChild.nodeValue

'Rodin'

In [80]:
data[-1]

{'time': 1649, 'value': 'England, yes.'}